In [2]:
!pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
from google.colab import drive
import plotly.express as px
from haversine import haversine

drive.mount('/content/drive')
df_raw = pd.read_csv('/content/drive/MyDrive/FTC/train.csv')
df_raw.head()


Mounted at /content/drive


,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30


In [4]:
df1 = df_raw.copy()


## Converting Data Type

In [5]:
# convering Delivery_person_Age
line_nan = (df1['Delivery_person_Age'] != 'NaN ') 
df1 = df1.loc[line_nan, :].copy()


line_nan = (df1['Road_traffic_density'] != 'NaN ') 
df1 = df1.loc[line_nan, :].copy()

df1['Delivery_person_Age'] = df1['Delivery_person_Age'].astype(int)

# convering Delivery_person_Ratings
df1['Delivery_person_Ratings'] = df1['Delivery_person_Ratings'].astype( float )

# converting Order_Date 
df1['Order_Date'] = pd.to_datetime(df1['Order_Date'], format = '%d-%m-%Y')

# convertig multiple_deliveries
aux = (df1['multiple_deliveries']!= 'NaN ')
df1 = df1.loc[aux, :].copy()

df1['multiple_deliveries'] = df1['multiple_deliveries'].astype( int )

# removing spaces in strings or objects
#df1 = df1.reset_index(drop = True)
#for i in range(len(df1)):
#   df1.loc[i,'ID'] = df1.loc[i, 'ID'].strip()

df1.loc[:,'ID']=df1.loc[:,'ID'].str.strip()
df1.loc[:,'Delivery_person_ID']=df1.loc[:,'Delivery_person_ID'].str.strip()
df1.loc[:,'Type_of_order']=df1.loc[:,'Type_of_order'].str.strip()
df1.loc[:,'Road_traffic_density']=df1.loc[:,'Road_traffic_density'].str.strip()
df1.loc[:,'Type_of_vehicle']=df1.loc[:,'Type_of_vehicle'].str.strip()
df1.loc[:,'Festival']=df1.loc[:,'Festival'].str.strip()
df1.loc[:,'City']=df1.loc[:,'City'].str.strip()

df1['Time_taken(min)'] = df1['Time_taken(min)'].apply( lambda x: x.split( '(min)')[1])
df1['Time_taken(min)'] = df1['Time_taken(min)'].astype( int )

In [6]:
df1.iloc[0,:]

ID                                          0x4607
Delivery_person_ID                  INDORES13DEL02
Delivery_person_Age                             37
Delivery_person_Ratings                        4.9
Restaurant_latitude                      22.745049
Restaurant_longitude                     75.892471
Delivery_location_latitude               22.765049
Delivery_location_longitude              75.912471
Order_Date                     2022-03-19 00:00:00
Time_Orderd                               11:30:00
Time_Order_picked                         11:45:00
Weatherconditions                 conditions Sunny
Road_traffic_density                          High
Vehicle_condition                                2
Type_of_order                                Snack
Type_of_vehicle                         motorcycle
multiple_deliveries                              0
Festival                                        No
City                                         Urban
Time_taken(min)                

## Quantidades de pedidos por dia

In [7]:
cols = ['ID', 'Order_Date']
df_aux = df1.loc[:, cols].groupby(['Order_Date']).count().reset_index()



In [8]:
px.bar(df_aux, x ='Order_Date', y = 'ID')

## Quantidade de pedidos por semana

In [9]:
# treanformando em semanas considerando primeir odia util seguna usar W se for Domingo usar U
df1['week_of_year'] = df1['Order_Date'].dt.strftime('%U')
df_aux = df1.loc[:, ['ID', 'week_of_year']].groupby(['week_of_year']).count().reset_index()

px.line(df_aux, x = 'week_of_year', y = 'ID')

In [10]:
df1.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,...,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min),week_of_year
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,...,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,24,11
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,...,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,33,12
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,...,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,26,11
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,...,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,21,14
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,...,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,30,12


## Distribuicao de pedidos por tipo de trafego

In [11]:
df_aux = df1.loc[:, ['ID', 'Road_traffic_density']].groupby(['Road_traffic_density']).count().reset_index()
df_aux = df_aux.loc[df_aux['Road_traffic_density']!= 'NaN', :]
df_aux['%_delivery'] = df_aux['ID'] / df_aux['ID'].sum()

#df_aux

px.pie(df_aux, values = '%_delivery', names = 'Road_traffic_density')


## Volume de pedidos por cidade e trafego

In [12]:
df_aux = df1.loc[:, ['ID', 'City', 'Road_traffic_density']].groupby(['City','Road_traffic_density']).count().reset_index()
df_aux = df_aux.loc[df_aux['City'] != 'NaN', :]
df_aux = df_aux.loc[df_aux['Road_traffic_density'] != 'NaN', :]

#df_aux

px.scatter(df_aux, x = 'City', y = 'Road_traffic_density', size = 'ID', color = 'City')

## quantidade de pedidos por entregador por semana

In [13]:
from numpy import inner
# quantidad de pedidos por semana/numero unico de entregadores por semana

df_aux1 = df1.loc[:, ['ID', 'week_of_year']].groupby(['week_of_year']).count().reset_index()
df_aux2 = df1.loc[:, ['Delivery_person_ID', 'week_of_year']].groupby('week_of_year').nunique().reset_index()

df_aux = pd.merge(df_aux1,df_aux2, how = 'inner' )
df_aux['order_by_delivey'] = df_aux['ID'] / df_aux['Delivery_person_ID']
#df_aux.head()

px.line(df_aux, x = 'week_of_year', y = 'order_by_delivey')

In [14]:
from folium.vector_layers import Popup
# localização central para cada tipo de tráfego
import folium

df_aux = df1.loc[:, ['City', 'Road_traffic_density', 'Delivery_location_latitude','Delivery_location_longitude']].groupby(['City', 'Road_traffic_density']).median().reset_index()
df_aux = df_aux.loc[df_aux['City'] != 'NaN', :]
df_aux = df_aux.loc[df_aux['Road_traffic_density'] != 'NaN', :]


map = folium.Map()

for index, location_info in df_aux.iterrows():
  folium.Marker([location_info['Delivery_location_latitude'],
                 location_info['Delivery_location_longitude']],
                 popup= location_info[['City', 'Road_traffic_density']]).add_to(map)
  



map

# Delivery vision




### A menor e maior idade dos entregadores



In [15]:
df1.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,...,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min),week_of_year
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,...,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,24,11
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,...,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,33,12
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,...,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,26,11
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,...,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,21,14
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,...,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,30,12


In [16]:
print('A maior idade dos entregadores e: ', df1.loc[:, 'Delivery_person_Age'].max())
print('A maior idade dos entregadores e: ', df1.loc[:, 'Delivery_person_Age'].min())

A maior idade dos entregadores e:  39
A maior idade dos entregadores e:  20


### Pior e melhor condicao de veiculo

In [17]:
print('A pior condicao do veiculo: ', df1.loc[:, 'Vehicle_condition'].max())
print('A pior condicao do veiculo: ', df1.loc[:, 'Vehicle_condition'].min())

A pior condicao do veiculo:  2
A pior condicao do veiculo:  0


### A avaliacao media por entregador

In [18]:
df_average_rating_per_deliver = df1.loc[:, ['Delivery_person_ID', 'Delivery_person_Ratings']].groupby(['Delivery_person_ID']).mean().reset_index()
df_average_rating_per_deliver.head()

,Delivery_person_ID,Delivery_person_Ratings
0,AGRRES010DEL01,4.761538
1,AGRRES010DEL02,4.671429
2,AGRRES010DEL03,4.575000
3,AGRRES01DEL01,4.522222
4,AGRRES01DEL02,4.700000


### A Avaliaca ,edia e o desvio padrao por tipo de trafego

In [19]:
#using one line
#df_average_rating_by_trafic = df1.loc[:, ['Delivery_person_Ratings','Road_traffic_density' ]].groupby(['Road_traffic_density']).agg({'Delivery_person_Ratings':['mean','std']})

#using two lines
df_avg_std_by_trafic = (df1.loc[:, ['Delivery_person_Ratings','Road_traffic_density' ]]
                              .groupby(['Road_traffic_density'])
                              .agg({'Delivery_person_Ratings':['mean','std']}))

#df_std_rating_by_trafic = df1.loc[:, ['Delivery_person_Ratings','Road_traffic_density' ]].groupby(['Road_traffic_density']).std().reset_index()

# Manipulation dataframe column name
df_avg_std_by_trafic.columns = ['delivery_mean', 'delivery_std']
# Manipulation dataframe format

df_avg_std_by_trafic.reset_index()

,Road_traffic_density,delivery_mean,delivery_std
0,High,4.653492,0.272312
1,Jam,4.595395,0.328749
2,Low,4.646780,0.335426
3,Medium,4.660627,0.273472


### A Avaliacao de media e desvio padrao por condicoes climaticas

In [20]:
#using one line
#df_average_rating_rating_by_wather = df1.loc[:, ['Delivery_person_Ratings','Weatherconditions' ]].groupby(['Weatherconditions']).agg({'Delivery_person_Ratings':['mean','std']})

#using two lines
df_avg_std_rating_by_wather = (df1.loc[:, ['Delivery_person_Ratings','Weatherconditions' ]]
                              .groupby(['Weatherconditions'])
                              .agg({'Delivery_person_Ratings':['mean','std']}))

#df_std_rating_rating_by_wather = df1.loc[:, ['Delivery_person_Ratings','Weatherconditions' ]].groupby(['Weatherconditions']).std().reset_index()

# Manipulation dataframe column name
df_avg_std_rating_by_wather.columns = ['delivery_mean', 'delivery_std']
# Manipulation dataframe format

df_avg_std_rating_by_wather.reset_index()

,Weatherconditions,delivery_mean,delivery_std
0,conditions Cloudy,4.652190,0.280269
1,conditions Fog,4.653549,0.273893
2,conditions Sandstorms,4.613000,0.309512
3,conditions Stormy,4.613369,0.311624
4,conditions Sunny,4.658030,0.393090
5,conditions Windy,4.617804,0.303464


### 10 entregadores mais rapidos da cidade



In [21]:
df2 = df1.loc[:, ['Delivery_person_ID', 'Time_taken(min)', 'City']].groupby(['City', 'Delivery_person_ID']).median().sort_values(['Time_taken(min)', 'City'], ascending = False).reset_index()

df_aux01 = df2.loc[df2['City'] == 'Metropolitian', : ].head(10)
df_aux02 = df2.loc[df2['City'] == 'Urban', : ].head(10)
df_aux03 = df2.loc[df2['City'] == 'Semi-Urban', : ].head(10)

pd.concat([df_aux01,df_aux02,df_aux03]).reset_index(drop = True)


,City,Delivery_person_ID,Time_taken(min)
0,Metropolitian,AURGRES11DEL03,41.5
1,Metropolitian,AGRRES02DEL01,40.0
2,Metropolitian,ALHRES18DEL02,39.0
3,Metropolitian,KOLRES03DEL03,39.0
4,Metropolitian,LUDHRES14DEL01,39.0
5,Metropolitian,LUDHRES17DEL03,39.0
6,Metropolitian,ALHRES03DEL01,38.5
7,Metropolitian,KOCRES20DEL01,38.5
8,Metropolitian,ALHRES12DEL01,37.5
9,Metropolitian,BHPRES19DEL01,37.0


### 10 entregadores mais lentos da cidade



In [22]:
df2 = df1.loc[:, ['Delivery_person_ID', 'Time_taken(min)', 'City']].groupby(['City', 'Delivery_person_ID']).mean().sort_values(['Time_taken(min)', 'City'], ascending = True).reset_index()

df_aux01 = df2.loc[df2['City'] == 'Metropolitian', : ].head(10)
df_aux02 = df2.loc[df2['City'] == 'Urban', : ].head(10)
df_aux03 = df2.loc[df2['City'] == 'Semi-Urban', : ].head(10)

pd.concat([df_aux01,df_aux02,df_aux03]).reset_index(drop = True)


,City,Delivery_person_ID,Time_taken(min)
0,Metropolitian,KNPRES03DEL02,15.750000
1,Metropolitian,ALHRES02DEL02,17.800000
2,Metropolitian,KNPRES01DEL01,19.125000
3,Metropolitian,KOLRES01DEL03,19.125000
4,Metropolitian,KOCRES02DEL02,19.250000
5,Metropolitian,KOCRES08DEL03,19.800000
6,Metropolitian,KOCRES16DEL03,20.000000
7,Metropolitian,KOCRES02DEL03,20.375000
8,Metropolitian,GOARES14DEL01,20.400000
9,Metropolitian,DEHRES20DEL03,20.428571


In [23]:
df1.iloc[0,:]

ID                                          0x4607
Delivery_person_ID                  INDORES13DEL02
Delivery_person_Age                             37
Delivery_person_Ratings                        4.9
Restaurant_latitude                      22.745049
Restaurant_longitude                     75.892471
Delivery_location_latitude               22.765049
Delivery_location_longitude              75.912471
Order_Date                     2022-03-19 00:00:00
Time_Orderd                               11:30:00
Time_Order_picked                         11:45:00
Weatherconditions                 conditions Sunny
Road_traffic_density                          High
Vehicle_condition                                2
Type_of_order                                Snack
Type_of_vehicle                         motorcycle
multiple_deliveries                              0
Festival                                        No
City                                         Urban
Time_taken(min)                

# Restaurant

## Quantidade de entregadores unicos

In [24]:
delivery_unique = len(df1.loc[:, 'Delivery_person_ID'].unique())
print('Numeros de entregadores unicos e: {}'.format(delivery_unique))

Numeros de entregadores unicos e: 1320


# Distancia media dos restaurantes e locais de entregue

In [25]:
#!pip install haversine

In [26]:
cols = ['Restaurant_longitude','Restaurant_latitude','Delivery_location_latitude', 'Restaurant_longitude']
df1.loc[:,col].



avg_distance

SyntaxError: ignored

In [ ]:
haversine()